In [ ]:
import json
import pandas as pd
import re

# Creating the Clean Test Data

## 1. Load Test Data

In [2]:
# Load the file once as a JSON array
with open("/Users/alex/Documents/Data Science Master/thesis_RAG/data/data_processed/Train_Val_Test/df_val.json", 'r') as f:
    data = json.load(f)  # not line-by-line!

# Convert list of dicts to DataFrame
df_test = pd.DataFrame(data)
df_test.head(10)

,ID,question,answer,context,gold_context,operation,source
0,UNP/2015/page_80.pdf,what percentage of total minimum lease payment...,68%,['we maintain and operate the assets based on ...,{'table_7': 'millions the total minimum lease ...,"add(3430, 1587), divide(3430, #0)",FinQA
1,CMCSA/2004/page_30.pdf,Q: what was the value of the long-term strateg...,551,['management 2019s discussion and analysis of ...,{'text_18': 'additions to intangibles during 2...,"add(250, 133), add(168, #0)",ConvFinQA
2,MRO/2011/page_37.pdf,by how much did the wti crude oil benchmark in...,53.2%,"['item 7 .', 'management 2019s discussion and ...",{'table_1': 'benchmark the wti crude oil ( dol...,"subtract(95.11, 62.09), divide(#0, 62.09)",FinQA
3,a862932d,Risk mgmt for capital alloc. in accelerated re...,The changes in the terms of the Series B Prefe...,"[In conjunction with the Garrett spin-off, the...",,None,FinDER
4,LMT/2012/page_44.pdf,"Q: what was the operating profit, in millions,...",7.4%,['aeronautics 2019 operating profit for 2011 i...,{'table_2': 'the operating profit of 2012 is 8...,"subtract(874, 814), divide(#0, 814)",ConvFinQA
5,6e5aff44,Revenue recognition timing for performance obl...,"Raymond James Financial, Inc. recognizes reven...",[Revenue from contracts with customers is reco...,,None,FinDER
6,C/2008/page_26.pdf,what percentage of incremental risk-weighted a...,4%,"['commitments .', 'for a further description o...",{'table_4': 'in billions of dollars the studen...,"divide(3.5, 98.9)",FinQA
7,STT/2017/page_63.pdf,Q: what was the value of the s&p500 index in 2...,53%,"[""state street corporation | 52 shareholder re...",{'table_2': 'the s&p 500 index of 2012 is 100 ...,"subtract(153, 100), divide(#0, 100)",ConvFinQA
8,ETR/2008/page_376.pdf,what is the net change in net revenue during 2...,-1.4,"['entergy texas , inc .', ""management's financ...",{'table_1': 'the 2007 net revenue of amount ( ...,"subtract(440.9, 442.3)",FinQA
9,7ec5e05d,2024 total opex: Nordson Corp (NDSN) cost-of-s...,"For 2024, the cost of sales is $1,203,792 and ...",[Consolidated Statements of Income\nYears ende...,,Addition,FinDER


## 2. Categorizing Test Data By Operation Type

In [ ]:
def hybrid_reasoning_classifier(row):
    op = str(row['operation']).lower()
    question = str(row['question']).lower()
    source = row['source']

    if pd.isna(op) or op.strip() == 'none':
        return 'Non-numerical'

    # FinDER: Use natural language tags but catch explicit math
    if source == 'FinDER':
        if 'compositional' in op or 'comparison' in op:
            return 'Compositional'
        elif any(kw in op for kw in ['multiply', 'division', 'addition', 'subtraction']):
            return 'Arithmetic'
        else:
            return 'Non-numerical'

    # ConvFinQA: Conversational → likely compositional
    if source == 'ConvFinQA':
        if question.count("q:") > 1:
            return 'Compositional'

    # Use regex to extract full operation calls like add(...), subtract(...), etc.
    base_ops = ['add', 'subtract', 'divide', 'multiply', 'average', 'max', 'min']
    ops = re.findall(r'\b(?:' + '|'.join(base_ops) + r')\s*\(.*?\)', op)

    used_ops = set()
    for o in ops:
        for fn in base_ops:
            if o.strip().startswith(fn):
                used_ops.add(fn)

    if len(used_ops) > 1:
        return 'Compositional'
    elif len(used_ops) == 1 and len(ops) > 1:
        return 'Compositional'
    else:
        return 'Arithmetic'

In [4]:
df_test['hybrid_reasoning'] = df_test.apply(hybrid_reasoning_classifier, axis=1)

In [5]:
df_test[
    (df_test['source'] == 'FinQA') &
    (df_test['operation'].str.contains('divide')) &
    (df_test['hybrid_reasoning'] == 'Arithmetic')
].head()

,ID,question,answer,context,gold_context,operation,source,hybrid_reasoning
6,C/2008/page_26.pdf,what percentage of incremental risk-weighted a...,4%,"['commitments .', 'for a further description o...",{'table_4': 'in billions of dollars the studen...,"divide(3.5, 98.9)",FinQA,Arithmetic
16,JPM/2013/page_132.pdf,what was the ratio of the firm 2019s cra loan ...,1.125,['management 2019s discussion and analysis 138...,"{'text_11': 'at december 31 , 2013 and 2012 , ...","divide(18, 16)",FinQA,Arithmetic
20,ADBE/2012/page_87.pdf,what is the yearly amortization rate related t...,10%,['goodwill is assigned to one or more reportin...,{'table_2': 'the customer contracts and relati...,"divide(const_100, 10)",FinQA,Arithmetic
29,ZBH/2004/page_65.pdf,what was the percentage change in accumulated ...,83%,"['z i m m e r h o l d i n g s , i n c .', 'a n...",{'table_1': 'the beginning balance at january ...,"divide(114.6, 138.7)",FinQA,Arithmetic
30,PKG/2006/page_27.pdf,what was the operating income margin for 2006?,10%,['results of operations year ended december 31...,{'table_1': '( in millions ) the net sales of ...,"divide(225.9, 2187.1)",FinQA,Arithmetic


In [87]:
df_test.groupby(['source', 'hybrid_reasoning']).size()

source     hybrid_reasoning
ConvFinQA  Compositional       530
FinDER     Arithmetic           25
           Compositional        52
           Non-numerical       493
FinQA      Arithmetic          368
           Compositional       252
dtype: int64

In [7]:
for reasoning in df_test['hybrid_reasoning'].unique():
    print(f"\n========== Reasoning Type: {reasoning.upper()} ==========\n")
    
    for source in df_test['source'].unique():
        subset = df_test[(df_test['source'] == source) & (df_test['hybrid_reasoning'] == reasoning)]

        if len(subset) == 0:
            continue

        print(f"\n--- Source: {source} ---")
        sample = subset.sample(n=min(3, len(subset)), random_state=42)

        for _, row in sample.iterrows():
            print(f"\nQ: {row['question']}")
            print(f"A: {row['answer']}")
            print(f"Operation: {row['operation']}")
            print(f"Gold Context: {row['gold_context']}")
            print("="*70)


========== Reasoning Type: COMPOSITIONAL ==========


--- Source: FinQA ---

Q: what is the growth rate in the risk-free interest rate from 2004 to 2005?
A: 38.7%
Operation: subtract(4.19, 3.02), divide(#0, 3.02)
Gold Context: {'table_3': 'the risk-free interest rate of 2006 is 4.60 ; the risk-free interest rate of 2005 is 4.19 ; the risk-free interest rate of 2004 is 3.02 ;'}

Q: what was the percent change in the value of commercial paper outstanding between 2010 and 2011?
A: 18%
Operation: subtract(2.80, 2.38), divide(#0, 2.38)
Gold Context: {'text_0': 'we maintain an effective universal shelf registration that allows for the public offering and sale of debt securities , capital securities , common stock , depositary shares and preferred stock , and warrants to purchase such securities , including any shares into which the preferred stock and depositary shares may be convertible , or any combination thereof .', 'text_5': 'at december 31 , 2011 , we had $ 2.38 billion of commercial 

# 3. Extend Sampling

In [ ]:
# Define sampling plan by (source, reasoning type)
sample_plan = {
    ('FinQA', 'Arithmetic'): 25,
    ('FinQA', 'Compositional'): 25,
    ('ConvFinQA', 'Compositional'): 30,
    ('FinDER', 'Arithmetic'): 20,
    ('FinDER', 'Compositional'): 15,
    ('FinDER', 'Non-numerical'): 35,
}

# Sample from df_test (your labeled full dataset)
samples = []
for (source, reasoning), count in sample_plan.items():
    subset = df_test[(df_test['source'] == source) & (df_test['hybrid_reasoning'] == reasoning)]
    if len(subset) < count:
        print(f"Not enough samples in ({source}, {reasoning}), taking {len(subset)} instead of {count}")
        sample = subset.sample(n=len(subset), random_state=42)
    else:
        sample = subset.sample(n=count, random_state=42)
    samples.append(sample)

# Combine
df_review = pd.concat(samples).reset_index(drop=True)

In [ ]:
# Step 1: Track used questions to avoid duplication
used_questions = set(df_review['question'])

# Step 2: Define additional sampling plan
extended_sample_plan = {
    ('FinQA', 'Arithmetic'): 10,
    ('FinQA', 'Compositional'): 10,
    ('ConvFinQA', 'Compositional'): 30,
}

# Step 3: Sample new questions avoiding duplicates
new_samples = []
for (source, reasoning), count in extended_sample_plan.items():
    subset = df_test[
        (df_test['source'] == source) &
        (df_test['hybrid_reasoning'] == reasoning) &
        (~df_test['question'].isin(used_questions))
    ]
    
    if len(subset) < count:
        print(f"Not enough available samples in ({source}, {reasoning}), taking {len(subset)} instead of {count}")
        sample = subset.sample(n=len(subset), random_state=42)
    else:
        sample = subset.sample(n=count, random_state=42)
    
    new_samples.append(sample)

# Step 4: Combine with original review dataset
df_new = pd.concat(new_samples).reset_index(drop=True)
df_review = pd.concat([df_review, df_new]).reset_index(drop=True)

# Optional sanity check
print(f"df_review now has {len(df_review)} entries.")

✅ df_review now has 200 entries.


In [ ]:
# Step 1: Identify duplicate IDs
duplicate_ids = df_review[df_review.duplicated(subset='ID', keep=False)]['ID'].unique()

# Step 2: Extract and drop duplicates (keep only the first occurrence)
duplicates = df_review[df_review['ID'].isin(duplicate_ids)]
df_review = df_review.drop_duplicates(subset='ID', keep='first').reset_index(drop=True)

# Step 3: Track used questions
used_questions = set(df_review['question']) | set(duplicates['question'])

# Step 4: Resample replacements for the dropped duplicates
replacements = []

for _, dup_row in duplicates.iterrows():
    source = dup_row['source']
    reasoning = dup_row['hybrid_reasoning']
    
    subset = df_test[
        (df_test['source'] == source) &
        (df_test['hybrid_reasoning'] == reasoning) &
        (~df_test['question'].isin(used_questions))
    ]
    
    if not subset.empty:
        replacement = subset.sample(n=1, random_state=None)
        replacements.append(replacement)
        used_questions.add(replacement['question'].values[0])
        print(f"Replacement added for ({source}, {reasoning})")
    else:
        print(f"No replacement available for ({source}, {reasoning})")

# Step 5: Add replacements back
if replacements:
    df_review = pd.concat([df_review] + replacements).reset_index(drop=True)

✅ Replacement added for (ConvFinQA, Compositional)
✅ Replacement added for (FinQA, Compositional)
✅ Replacement added for (FinQA, Compositional)
✅ Replacement added for (FinQA, Arithmetic)
✅ Replacement added for (FinQA, Arithmetic)
✅ Replacement added for (FinQA, Arithmetic)
✅ Replacement added for (ConvFinQA, Compositional)
✅ Replacement added for (ConvFinQA, Compositional)
✅ Replacement added for (FinQA, Compositional)
✅ Replacement added for (ConvFinQA, Compositional)


In [78]:
for i, row in df_review.iterrows():
    print(f"\n[{i}] - Source: {row['source']} | Reasoning: {row['hybrid_reasoning']}")
    print(f"Q: {row['question']}")
    print(f"A: {row['answer']}")
    print(f"Operation: {row['operation']}")
    print(f"Gold Context: {row['gold_context']}")
    print("=" * 80)
 


[0] - Source: FinDER | Reasoning: Arithmetic
Q: GM operating margin 2023 vs 2022, GM.
A: To calculate the operating profit margin, we divide Operating Income by Total Net Sales and Revenue. For 2023, the calculation is as follows:

• 2023 Operating Profit Margin = 9,298 / 171,842 ≈ 0.0541, or about 5.41%.

For 2022, using the same method:

• 2022 Operating Profit Margin = 10,315 / 156,735 ≈ 0.0658, or about 6.58%.

This comparison shows that the operating margin declined from approximately 6.58% in 2022 to about 5.41% in 2023.
Operation: Division
Gold Context: 

[1] - Source: FinQA | Reasoning: Arithmetic
Q: what percentage of incremental risk-weighted assets are student loans at january 1 , 2010?
A: 4%
Operation: divide(3.5, 98.9)
Gold Context: {'table_4': 'in billions of dollars the student loans of incremental gaap assets is 14.4 ; the student loans of incremental risk- weighted assets is 3.5 ;', 'table_8': 'in billions of dollars the total of incremental gaap assets is $ 179.0 ; t

In [80]:
df_review.head(10)

,ID,question,answer,context,gold_context,operation,source,hybrid_reasoning,answer_verified,context_verified,notes,question_clean
0,182f0809,"GM operating margin 2023 vs 2022, GM.","To calculate the operating profit margin, we d...","[CONSOLIDATED INCOME STATEMENTS\n(In millions,...",,Division,FinDER,Arithmetic,,,,NaN
1,C/2008/page_26.pdf,what percentage of incremental risk-weighted a...,4%,"['commitments .', 'for a further description o...",{'table_4': 'in billions of dollars the studen...,"divide(3.5, 98.9)",FinQA,Arithmetic,,,,NaN
2,ETR/2004/page_20.pdf,what is the growth rate in net revenue in 2003...,0.1%,"[""entergy corporation and subsidiaries managem...",{'table_1': 'the 2002 net revenue of ( in mill...,"subtract(4214.5, 4209.6), divide(#0, 4209.6)",FinQA,Compositional,,,,NaN
3,c593f878,NGC's cyber investments boost investor confide...,The provided information allows us to assess h...,[We recognize the critical importance of maint...,,None,FinDER,Non-numerical,,,,NaN
4,cb08b8b0,"SBA Comm., credit evals & DTA quality receivab...",The details provided illustrate that SBA Commu...,[Site leasing revenues\n\nRevenue from site le...,,None,FinDER,Non-numerical,,,,NaN
5,GPN/2009/page_85.pdf,Q: what was the fair value of share awards ves...,265% increase,['notes to consolidated financial statements 2...,{'text_2': 'the total fair value of share awar...,"subtract(6.2, 1.7), divide(#0, 1.7)",ConvFinQA,Compositional,,,,NaN
6,AAPL/2012/page_36.pdf,Q: what was the difference between the net sal...,66%,['$ 43.3 million in 2011 compared to $ 34.1 mi...,{'table_1': 'the net sales of 2012 is $ 156508...,"subtract(108249, 65225), divide(#0, 65225)",ConvFinQA,Compositional,,,,NaN
7,AMT/2012/page_123.pdf,what was the cost per tower in the colombia mo...,856067,['american tower corporation and subsidiaries ...,"{'text_8': '( 201ccolombia movil 201d ) , wher...","multiply(182.0, const_1000000), divide(#0, 2126)",FinQA,Compositional,,,,NaN
8,C/2008/page_212.pdf,Q: what was the fair value of the msr in 2008?...,-33.2%,['the company has elected the fair-value optio...,{'table_1': 'in millions of dollars the carryi...,"subtract(4273, 6392), divide(#0, 6392)",ConvFinQA,Compositional,,,,NaN
9,ETR/2016/page_175.pdf,Q: what is the implicit interest cost rate?\nA...,13225.7,['entergy corporation and subsidiaries notes t...,{'table_6': 'the years thereafter of amount ( ...,"divide(5.13, const_100), multiply(#0, 257812)",ConvFinQA,Compositional,,,,NaN


In [ ]:
# Snapshot by (Source, Reasoning Type)
snapshot = df_review.groupby(['source', 'hybrid_reasoning']).size().reset_index(name='count')

# Pivot for cleaner table view
pivot_snapshot = snapshot.pivot(index='source', columns='hybrid_reasoning', values='count').fillna(0).astype(int)

# Add total per source
pivot_snapshot['Total'] = pivot_snapshot.sum(axis=1)

# Add grand total at the bottom
pivot_snapshot.loc['Total'] = pivot_snapshot.sum()

# Display
import IPython.display as disp
disp.display(pivot_snapshot)

hybrid_reasoning,Arithmetic,Compositional,Non-numerical,Total
source,,,,
ConvFinQA,0,60,0,60
FinDER,20,15,35,70
FinQA,37,37,0,74
Total,57,112,35,204


In [82]:
# Save the cleaned and reviewed DataFrame as JSON (records format)
df_review.to_json("gold_test_data.json", orient="records", indent=2, force_ascii=False)

In [86]:
# Drop unwanted columns
df_cleaned = df_review.drop(
    columns=[
        "answer_verified",
        "context_verified",
        "notes",
        "question_clean"
    ],
    errors="ignore"  # prevents crash if any column doesn't exist
)

# Rename column
df_cleaned = df_cleaned.rename(columns={"hybrid_reasoning": "reasoning_type"})

# Export cleaned version
output_path = "/Users/alex/Documents/Data Science Master/thesis_RAG/data/data_processed/Train_Val_Test/gold_test_data.json"

df_cleaned.to_json(
    output_path,
    orient="records",
    indent=2,
    force_ascii=False
)

print(f"Cleaned gold test data saved to: {output_path}")

Cleaned gold test data saved to: /Users/alex/Documents/Data Science Master/thesis_RAG/data/data_processed/Train_Val_Test/gold_test_data.json
